In [ ]:
# instalar Parquet, Pandas e Pyarrow
%!pip install parquet
%!pip install pandas
%!pip install pyarrow

In [ ]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import os